# Prueba tecnica parte 2: Limpieza y análisis exploratorio
A continuacion se muestra el proceso realizado para la limpienza y normalización de datos del dataset: `datos mercado inmobiliario`

In [ ]:
# Importar librerias que se van a usar y cargar el DataFrame desde el CSV
import pandas as pd
import re
import matplotlib.pyplot as plt

path = r"C:\Users\LENOVO\OneDrive\src\Analisis_de_mercado\datos_prueba_técnica- Data.csv"
## Cambiar ruta por la del bucket
df = pd.read_csv(path, encoding='utf-8')
print(df.head())


## 1. Analisis de celdas duplicadas
pandas tiene la función `df.duplicated` que analiza si una fila se encontro anteriormente en el df. Se va a usar esta función analizando 5 columnas que son criticas en la coincidencia de los registros: 
* id 
* codigo_web
* tipo de inmueble: Este puede estar listado de otra forma edificio o casa 
* nombre_publicación
* last_edited: Esto se realiza con el fin de conservar la versión final de cada registro en caso de estar duplicado. Se realiza un sort para tener el ultimo registro al final

In [4]:
# Sort de las filas para tener el ultimo update abajo siempre
df = df.sort_values('last_edited', ascending=True)
mask = df.duplicated(subset=(["id","codigo_web","tipo_inmueble","nombre_publicacion","last_edited"]))
# Print para contar el numero de propiedades que tienen un registro duplicado, cuenta los TRUE del mask
print(f"Cantidad de propiedades duplicadas: {mask.sum()}") 
# Eliminar columnas duplicadas
df = df.drop_duplicates(subset=["id","codigo_web","tipo_inmueble","nombre_publicacion","last_edited"])
print(f"propiedades totales: {df["id"].count()}")


Cantidad de propiedades duplicadas: 69999
propiedades totales: 70001


## 2. Normalizado de columnas de TEXTO 
### Revisar que columnas necesitan ser normalizadas
para determinar cuales columnas tienen texto con formatos diferentes, se realiza un analisis preliminar empleando la función `set()`

In [5]:
ciudad = set(df["ciudad"]) 
portal_inm = set(df["portal_inmobiliario"]) 
tipo_inmueble = set(df["tipo_inmueble"]) 
tipo_negocio = set(df["tipo_negocio"]) 
departamento = set(df["departamento"])
subzona = sorted(set(df["subzona"]))

print(f"{ciudad}\n{portal_inm}\n{tipo_inmueble}\n{tipo_negocio}\n{departamento}\n{subzona}")


{'bogota', 'bogotá D.C', 'BOGOTÁ', 'bogotá dc.', 'bogota d.c.', 'Bogotá', 'Bogota DC'}
{'Metro Cuadrado', 'CienCuadras', 'Habi', 'Fincaraiz', 'MercadoLibre'}
{'Apartamento', 'APARTAMENTO', 'Casa'}
{'FOR_SALE'}
{'Cundinamarca', 'CUNDINAMARCA'}
['Antonio Nariño', 'Barrios Unidos', 'Bosa', 'Chapinero', 'Ciudad Bolivar', 'Engativá', 'Fontibón', 'Kennedy', 'La Candelaria', 'Los Mártires', 'Puente Aranda', 'Rafael Uribe', 'San Cristóbal', 'Santa Fé', 'Suba', 'Sumapaz', 'Teusaquillo', 'Tunjuelito', 'Usaquén', 'Usme']


la columna `ciudad, tipo_inmueble, departamento` tienen inconsistencias de formato, `portal_inmobiliario, subzona` no tienen entonces solo se va a trabajar con las primeras 3. 
### Ciudad
Para la columna `[ciudad]` se va a eliminar los espacios, se va a dejar en mayusculas y eliminar los acentos y la palabra "DC" solo conservando **"BOGOTA"** 
### tipo de inmueble & departamento
Para estos, como no hay acentos ni condiciones especiales, solo se van a dejar en mayusculas

In [6]:
df['ciudad'] = df['ciudad'].str.upper().str.strip()
patron_ciudad = r"^(?=.*[AÁÉÍÓÚÜÑ]|\bD\.?C\.?\b).*$"
df['ciudad'] = df['ciudad'].str.replace(
    patron_ciudad,
    'BOGOTA',
    regex=True
)
df["tipo_inmueble"] = df["tipo_inmueble"].str.upper()
df["departamento"] = df["departamento"].str.upper()

# Se realiza de nuevo un check para comprobar que haya funcionado
ciudad = set(df["ciudad"]) 
tipo_inmueble = set(df["tipo_inmueble"]) 
departamento = set(df["departamento"]) 

print(f"{ciudad}\n{tipo_inmueble}\n{departamento}")


{'BOGOTA'}
{'CASA', 'APARTAMENTO'}
{'CUNDINAMARCA'}


## 3. Normalizado de columnas de FECHA 
Ahora se cambia el formato de las columnas que tienen tipo date(), para esto se usa la función `to_datetime()` que segun el contenido de texto se ajusta a un formato `%Y-%m-%d` o `%Y-%m-%d %H:%M:%S.%f'` o se autodetecta, para las columnas `"fecha_insercion_third_party",'fecha_publicacion', 'last_edited'` se realiza una funcion lambda y para `'fecha_insercion_interna'` se usa solo la funcion normal `to_datetime()`

In [ ]:
dates = ["fecha_insercion_third_party",'fecha_publicacion', 'last_edited']
df[dates] = df[dates].apply(lambda col: pd.to_datetime(col, format='%Y-%m-%d', errors='coerce'))
df['fecha_insercion_interna'] = pd.to_datetime(df['fecha_insercion_interna'],format='%Y-%m-%d %H:%M:%S.%f',errors='coerce')

print(df.info())


## 4. Limpieza de datos atipicos 
Primero se debe realizar una analisis sobre el conjunto de datos para estimar que datos no tienen sentido fisico, con eso se acotan los resultados y posteriormente eliminar los outliers o valores "atipicos". Esto se realiza con la función `.describe().T`


In [ ]:
print(df.shape[0]) # Tamaño de la matriz antes de la limpieza
# Se realiza el analisis sobre las columnas que tiene valores numericos sobre los cuales se puedan estimar limites "razonables"
col_numericas = ['latitud', 'longitud','area', 'area_total','habitaciones', 'banios', 'parqueaderos', 'precio_venta', 'precio_usd',
       'estrato', 'precio_admon','precio_admon_incluido', 'anhio_construccion', 'pisos_edificio', 'piso', 'ascensor',]
print(df[col_numericas].describe().T)


## 4.1. Valores sin sentido fisico
Al analizar el resultado de `.describe()` se pueden concluir las siguientes cosas: 
* Latitud y longitud estan fuera de los rangos para la ciudad de bogota
* Los precios de venta tienen valores muy pequeños - pero pueden ser no irreales
* El año de construcción esta mal listado el minimo no puede ser 2
* Los estratos en bogotá van de 1 a 6 
* El edificio mas alto en bogota tiene 67 pisos y es el bacata

Con esto se establecen los limites reales de algunas columnas y se realizan un filtrado del df.


In [10]:
latitud_bogota = [4.4,4.86] #lb, ub
longitud_bogota = [-76.63542,-73.97] #lb,ub
anhio_edificacion = [1580,2025] #año donde se fundo el Palacio de San Carlos
estratos = [1,6] #lb, ub
pisos_max = 67.0

mask = (
    (df['latitud'] >= latitud_bogota[0]) & (df['latitud'] <= latitud_bogota[1]) &
    (df['longitud'] >= longitud_bogota[0]) & (df['longitud'] <= longitud_bogota[1]) &
    (df['anhio_construccion'] >= anhio_edificacion[0]) & (df['anhio_construccion'] <= anhio_edificacion[1]) &
    (df['estrato'] >= estratos[0]) & (df['estrato'] <= estratos[1]) &
    (df["piso"]<= pisos_max)
)
# Aplicar la máscara al DataFrame
df_filtrado = df[mask]
print(f"Elementos filtrados: {df.shape[0]-df_filtrado.shape[0]}")
# Se hace print de la descripcion del nuevo df.
print(df_filtrado[col_numericas].describe().T)


Elementos filtrados: 52087
                      count          mean           std           min  \
latitud             17914.0  4.681592e+00  5.011399e-02  4.451515e+00   
longitud            17914.0 -7.408048e+01  4.378083e-02 -7.421353e+01   
area                17914.0  1.515140e+02  1.986082e+03  3.000000e+00   
area_total            313.0  4.586559e+02  6.592879e+02  1.800000e+01   
habitaciones        17914.0  3.008987e+00  1.448873e+00  1.000000e+00   
banios              17914.0  2.587027e+00  1.278476e+00  1.000000e+00   
parqueaderos        17914.0  1.342860e+00  1.194064e+00  0.000000e+00   
precio_venta        17914.0  1.870522e+09  5.927013e+10  1.150000e+06   
precio_usd          17913.0  4.559961e+05  1.446060e+07  2.766200e+02   
estrato             17914.0  4.138718e+00  1.346021e+00  1.000000e+00   
precio_admon        13460.0  4.397840e+06  7.073517e+07  1.000000e+00   
anhio_construccion  17914.0  2.006900e+03  9.625897e+00  1.937000e+03   
pisos_edificio        55

## 4.2. analisis de outliers 
Ahora se realiza un analisis outliers empleando el metodo de Tukey de percentiles y se va a realizar sobre las columnas `"precio_venta", "area",'precio_admon'` las cuales son datos relevantes y que se pueden modelar sin tener necesariamente "valores irreales".
1. Se realiza una vizualización prerliminar de los datos por dos medios. Primero un histograma y segundo un diagrama de cajas y bigotes
2. Se realiza el metodo de cuartiles para eliminar filtrar los outliers 
3. Se realiza la comparacion de los datos obtenidos finalmente

El analisis se realiza sobre el nuevo dataframe filtrado

In [ ]:
col_visual = ['latitud', 'longitud','area', 'area_total','habitaciones', 'banios', 'parqueaderos', 'precio_venta', 'precio_usd',
       'estrato', 'precio_admon','precio_admon_incluido', 'anhio_construccion', 'pisos_edificio', 'piso', 'ascensor']
col_estadisticos = ["precio_venta", "area",'precio_admon']

df_filtrado[col_visual].hist(bins=50, figsize=(16,12))
plt.show()


In [ ]:
df_filtrado[col_estadisticos].plot(kind="box",figsize=(16,12))
plt.xticks(rotation=90)
plt.show()


Se crea la función `eliminar_atipicos` que recibe como argumentos el df y las columnas sobre las cuales se va a realizar el analisis de outliers. Para el analisis de Tukey se calcular el _Q1_ y _Q3_ y con ellos el _IQR_ que es:
```
IQR = Q3 - Q1

donde se tienen que estimar los limites inferiores o superiores (bigotes)
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR
```


In [13]:
def eliminar_atipicos(df,columnas=col_estadisticos):
   # Se hace un if en caso de que no se entregue el argumento columnas
   if columnas is None:
       columnas = df.select_dtypes(include=["number"]).columns
   
   # Se crea una copia del df sobre el que se va a trabajar
   df_sin_atipicos = df.copy()
   
   for column in columnas:
       Q1 = df[column].quantile(0.25)
       Q3 = df[column].quantile(0.75)
       IQR = Q3 - Q1
       lower_b = Q1 - 1.5*IQR
       upper_b = Q3 + 1.5*IQR
       df_sin_atipicos = df_sin_atipicos[(df_sin_atipicos[column]>= lower_b) & (df_sin_atipicos[column]<= upper_b)]
   return df_sin_atipicos



Se aplica la funcion sobre el dataframe
Se analiza la opcion de aplicar la función por separado. 

1. Se realiza el analisis de outliers sobre todas las columnas

In [14]:
for i in range(3):
    df_filtro_completo =eliminar_atipicos(df_filtrado,columnas=col_estadisticos)
print("antes",df_filtrado.shape[0])
print("despues",df_filtro_completo.shape[0])
print(f"Elementos filtrados: {df_filtrado.shape[0]-df_filtro_completo.shape[0]}")


antes 17914
despues 12183
Elementos filtrados: 5731


In [ ]:
df_filtro_completo.plot(kind="box",figsize=(16,12))
plt.xticks(rotation=90)
plt.show()


In [ ]:
df_filtro_completo.hist(bins=50, figsize=(10,10))
plt.show()


Ahora se analiza el caso donde se realizan por separado la aplicación del analisis de outliers 

In [17]:
# Analisis sobre "area" y "precio_admon"
for i in range(3):
    new_df =eliminar_atipicos(df_filtrado,columnas=col_estadisticos[1:3])
# Analissi sobre el "precio_venta"
for i in range(3):
    new_df =eliminar_atipicos(new_df,columnas=col_estadisticos[0:1])
print("antes",df_filtrado.shape[0])
print("despues",new_df.shape[0])
print(f"Elementos filtrados: {df_filtrado.shape[0]-new_df.shape[0]}")


antes 17914
despues 11265
Elementos filtrados: 6649


In [ ]:
new_df.plot(kind="box",figsize=(16,12))
plt.xticks(rotation=90)
plt.show()


In [ ]:
new_df.hist(bins=50, figsize=(10,10))
plt.show()


## 5. Analisis de datos filtrados
Despues de observar los datos filtrados se obtuvieron las siguientes conclusiones 
1. Se partio con un total de 140.000 registros de los cuales **69999** son registros duplicados. Aproximadamente el **50%**. 
2. De los 70001 registros restantes se realizo un analisis sobre los maximos y minimos para encontrar anomalias. Se evidenciaron en las columnas de longitud, latitud, año de construcción, estratos y numero de piso. Sobre estos se ajustaron los valores de acuerdo a datos reales y que no son inconsistentes y se elimino un total de **52087** registros. Estos pueden tener uno o mas inputs que pueden ser producto de "error" humano sin embargo para el posterior analisis se descartan. Se elimino cualquier registros que no estuviera al menos en uno de los rangos permitidos. El total de registros despues de esto fue de **17914**
3. Se realizo un analisis de outliers para determinar valores "atipicos" en variables que pueden llegar a tener sentido fisico como un area o precio de venta elevados pero que pueden ser considerados excepcionales. Se empleo el metodo de cuartiles de Tukey para esto, y ocon ello se pudo filtrar un total de **6649** registros. 

Con esto se eliminaron al final **128.735** registros que representan el **91,95%** de los registros iniciales.

# Prueba tecnica parte 3: Análisis de datos
Para esta sección se quiere determinar si el precio de un inmueble varia según el piso en el que se encuentre. Para esto solo se van a contrastar dos variables **Precio de venta** y **piso**. Sin embargo, en necesario contemplar otras variables de forma no directa como es el estrato, area ya que estas influyen en gran medida sobre el precio. Para contemplar su impacto se va a realizar un analisis segmentado tomando por estrato y por cuartiles sobre el rango de areas y así tener grupos mas homogeneos. 
1. Se crea un nuevo DataFrame para el analisis
2. se eliminan celdas nulas
3. Se realiza la segmentación de las variables que no participan en la regreción
4. Se realiza una regrecion por cada combinacion de segmentos de variable definidos anteriormente

In [20]:
import statsmodels.api as sm
import numpy as np

# Selección de columnas para analisis estadistico y limpieza
df_model = new_df[['precio_venta', 'piso', 'area', 'habitaciones', 'estrato']].copy()
df_model.replace([np.inf, -np.inf], np.nan, inplace=True)
df_model.dropna(subset=['precio_venta', 'piso', 'area', 'habitaciones', 'estrato'], inplace=True)

# Segmentación de variables que no participan en la regreción
# Área en 4 cuantiles
df_model['area_seg'] = pd.qcut(
    df_model['area'], q=4, labels=['Q1','Q2','Q3','Q4']
)

# Estrato discretizado 1 a 6, intervalos (0.5,1.5], (1.5,2.5], ..., (5.5,6.5]
bins_estrato = np.arange(0.5, 7, 1)
labels_estrato = [1, 2, 3, 4, 5, 6]
df_model['estrato_seg'] = pd.cut(
    df_model['estrato'],
    bins=bins_estrato,
    labels=labels_estrato,
    right=True,
    include_lowest=True
)


In [ ]:
## Regreciones

results = []
#group_cols = ['area_seg', 'hab_seg', 'estrato_seg']
group_cols = ['area_seg', 'estrato_seg']

for keys, group in df_model.groupby(group_cols,observed=True):
    if len(group) < 100: #Se toman los grupos con mas de 100 propiedades para considerarlos significativo
        continue
    X = sm.add_constant(group['piso'])
    y = group['precio_venta']
    model = sm.OLS(y, X).fit()
    results.append({
        **dict(zip(group_cols, keys)),
        'coef_piso': model.params['piso'],
        'pval_piso': model.pvalues['piso'],
        'r2':        model.rsquared,
        'n_obs':     len(group)
    })

# 4. Resumen de resultados
summary = pd.DataFrame(results)
print(summary)


   area_seg  estrato_seg     coef_piso     pval_piso        r2  n_obs
0        Q1            2  2.417730e+06  7.648089e-16  0.084641    737
1        Q1            3  8.758614e+05  5.664901e-06  0.014896   1375
2        Q1            4 -3.607760e+05  4.282588e-01  0.001284    491
3        Q2            2  1.159411e+06  2.124149e-02  0.017110    310
4        Q2            3  1.261567e+06  9.024326e-05  0.012574   1214
5        Q2            4  2.408028e+06  2.677815e-06  0.024687    884
6        Q2            5  6.988555e+06  5.125543e-04  0.044278    269
7        Q2            6  2.897806e+06  1.168249e-01  0.010752    230
8        Q3            3  6.389254e+06  3.808924e-10  0.095529    393
9        Q3            4  7.093997e+06  6.412648e-26  0.079020   1349
10       Q3            5  1.555737e+07  3.118673e-11  0.088932    476
11       Q3            6  1.507429e+07  7.161817e-05  0.033517    465
12       Q4            3  7.860079e+06  9.676875e-02  0.023005    121
13       Q4         

## Analisis final
H_0 : El numero de piso **no** tiene efecto en el precio de venta

### Segmentos significativos (p-val < 0.05)
Se obtuvo que: Q1 (2,3), Q2 (2,3,4,5), Q3 (3,4,5,6), Q4 (4,5,6) Tuvieron valores significativos, hay una tendencia que indica que entre mayor sea el Q* (mas area) tiene a ser mas relevante a medida que se aumenta los estratos y los coeficientes aumentan a medida que se aumenta el estrato ya que el precio por apartamento es mayor. Todos presentan pendientes positivas lo cual el precio por apartamento aumenta a medida que aumenta el piso en el que se encuentra mostrando una relacion directamente proporcional 

### Segmentos no significativos (p-val >= 0.05)
solo 3 grupos tienen un p-val mayor al 5% Q1(4), Q2(6), Q4(3) con solo el primer caso con un coeficiente negativo

### R^2 
En la  tabla global, se tiene que el `R^2` mas alto es de 0.0955 lo cual es menor a 2. Esto indica que el piso no es un factor significativo al momento de estimar el precio de un inmueble, hay factores que pueden llegar a ser relevantes o el conjunto de estos. 

### Conclución 
Se puede asegurar en la mayoria de los casos estudiados con un 95% de seguridad que el piso en el que se encuenrtre un inmueble afecta el precio de venta de este. Sin embargo, este parece no ser el factor fundamental por el cual el precio fluctue, puede haber otros aspectos mas relevantes a analizar como el estrato, nivel de construcción, año de construcción, entre otros.  